In [1]:
import os
import numpy as np
from raug.metrics import Metrics, accuracy
from raug.utils.loader import get_labels_frequency
import pandas as pd

# Especificar o caminho para a pasta "best_results"
folder_path = "best_results"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
true_labels = pd.read_csv(_csv_path_test).sort_values(by=['image_id'])['category']

# Inicializar as variáveis para armazenar as previsões dos modelos
ensemble_preds_best = None
ensemble_preds_last = None
num_models = 0
ser_lab_freq = get_labels_frequency(_csv_path_test, "category", "image_id")
_labels_name = ser_lab_freq.index.values #np.reshape(ser_lab_freq.index.values, (ser_lab_freq.index.values.shape[0], 1))

df_best = pd.DataFrame()
df_last = pd.DataFrame()

# Percorrer as pastas de cada experimento
for experiment_folder in os.listdir(folder_path):

    if ".csv" in experiment_folder:
        continue

    print("Experimento: ", experiment_folder)

    _metric_options_best = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/best'),
        'pred_name_scores': 'predictions.csv',
    }
    _metric_options_last = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/last'),
        'pred_name_scores': 'predictions.csv',
    }


    experiment_path = os.path.join(folder_path, experiment_folder)
    metrics_best = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_best)
    metrics_last = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_last)

    ensemble_preds_best = None
    ensemble_preds_last = None

    model_folders = os.listdir(experiment_path)

    # Check if substring "senet154" is present in model_folders
    if any("senet154" in s for s in model_folders):
        continue

    
    # Cria path para o experimento atual
    if 'ensemble_average' in model_folders:
        model_folders.remove('ensemble_average')
    else:
        os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_average'))
        os.mkdir(os.path.join(_metric_options_best['save_all_path']))
        os.mkdir(os.path.join(_metric_options_last['save_all_path']))

    if 'ensemble_3_best_average' in model_folders:
        model_folders.remove('ensemble_3_best_average')

    try:

        # Percorrer as pastas de cada modelo
        for model_folder in model_folders:

            model_path = os.path.join(experiment_path, model_folder)

            # Carregar as previsões do modelo atual
            model_preds_path_best = os.path.join(model_path, "test_pred_best/predictions.csv")
            model_preds_path_last = os.path.join(model_path, "test_pred_last/predictions.csv")

            model_preds_best = pd.read_csv(model_preds_path_best).sort_values(by=['image'])
            model_preds_last = pd.read_csv(model_preds_path_last).sort_values(by=['image'])

            images_id = model_preds_best[['image']]
            model_preds_best = model_preds_best[['0','1','2']]
            model_preds_last = model_preds_last[['0','1','2']]

            # Adicionar as previsões do modelo atual ao ensemble
            if ensemble_preds_best is None:
                ensemble_preds_best = model_preds_best
            else:
                ensemble_preds_best += model_preds_best

            if ensemble_preds_last is None:
                ensemble_preds_last = model_preds_last
            else:
                ensemble_preds_last += model_preds_last

            num_models += 1

        # Calcular a média das previsões dos modelos
        ensemble_preds_best /= num_models
        ensemble_preds_last /= num_models

        metrics_best.update_scores(true_labels, ensemble_preds_best.to_numpy(), images_id.to_numpy())
        metrics_last.update_scores(true_labels, ensemble_preds_last.to_numpy(), images_id.to_numpy())

        metrics_best.compute_metrics()
        metrics_last.compute_metrics()

        metrics_best.save_scores()
        metrics_last.save_scores()

        folder_name = {'folder_name': experiment_folder}
        dict_best = {**folder_name, **metrics_best.metrics_values}
        dict_last = {**folder_name, **metrics_last.metrics_values}

        del dict_best['auc_and_roc_curve']
        del dict_best['precision_recall_report']

        del dict_last['auc_and_roc_curve']
        del dict_last['precision_recall_report']

        df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
        df_last = df_last.append(pd.DataFrame(dict_last, columns=dict_last.keys(), index=[0]), ignore_index=True)

    except Exception as e:
        print("Erro no experimento ", experiment_folder)
        print(e)


df_best.to_csv(os.path.join(folder_path, 'ensemble_average_best.csv'), index=False)
df_last.to_csv(os.path.join(folder_path, 'ensemble_average_last.csv'), index=False)

# Avaliar a precisão do modelo de ensemble
# true_classes_path = os.path.join(folder_path, "true_classes.csv")
# true_classes = np.loadtxt(_csv_path_test, delimiter=",", skiprows=1)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# accuracy = accuracy_score(true_classes, ensemble_classes)
# precision = precision_score(true_classes, ensemble_classes, average="macro")
# recall = recall_score(true_classes, ensemble_classes, average="macro")
# f1 = f1_score(true_classes, ensemble_classes, average="macro")

# # Imprimir as métricas de avaliação
# print(f"Acurácia: {accuracy:.3f}")
# print(f"Precisão: {precision:.3f}")
# print(f"Recall: {recall:.3f}")
# print(f"F1-score: {f1:.3f}")


Experimento:  dull_razor_shades_of_gray_cropped_images_folder_2
Experimento:  DA0_noPP
Experimento:  DA0_PP
Saving the scores in best_results/DA0_PP/ensemble_average/best
Saving the scores in best_results/DA0_PP/ensemble_average/last
Experimento:  dull_razor
Experimento:  DA1_noPP
Experimento:  shades_of_gray_cropped_images_folder
Experimento:  dull_razor_shades_of_gray_cropped_images_folder
Experimento:  DA3_noPP
Experimento:  dull_razor_shades_of_gray
Experimento:  DA0_noPP_3
Saving the scores in best_results/DA0_noPP_3/ensemble_average/best
Saving the scores in best_results/DA0_noPP_3/ensemble_average/last
Experimento:  dull_razor_2
Experimento:  DA-2_shades_of_gray_cropped_images_folder_150_epochs
Saving the scores in best_results/DA-2_shades_of_gray_cropped_images_folder_150_epochs/ensemble_average/best
Saving the scores in best_results/DA-2_shades_of_gray_cropped_images_folder_150_epochs/ensemble_average/last
Experimento:  DA2_cropped_images_folder_150_epochs_lr_init_0001
Saving 

/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/raug/utils/classification_metrics.py:347: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Saving the scores in best_results/shades_of_gray_2/ensemble_average/best
Saving the scores in best_results/shades_of_gray_2/ensemble_average/last
Experimento:  noDA_noPP
Experimento:  shades_of_gray
Experimento:  mean_filter
Experimento:  mean_filter_2
Experimento:  noDA_noPP_2
Saving the scores in best_results/noDA_noPP_2/ensemble_average/best
Saving the scores in best_results/noDA_noPP_2/ensemble_average/last
Experimento:  DA0_PPcropped
Experimento:  DA2_shades_of_gray_CLAHE_cropped_images_folder
Saving the scores in best_results/DA2_shades_of_gray_CLAHE_cropped_images_folder/ensemble_average/best
Saving the scores in best_results/DA2_shades_of_gray_CLAHE_cropped_images_folder/ensemble_average/last
Experimento:  noDA_PPcropped
Experimento:  CLAHE


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [2]:
import os
import numpy as np
from raug.metrics import Metrics, accuracy
from raug.utils.loader import get_labels_frequency
import pandas as pd
from raug.utils import classification_metrics as cmet

# Especificar o caminho para a pasta "best_results"
folder_path = "best_results"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
true_labels = pd.read_csv(_csv_path_test).sort_values(by=['image_id'])['category']

# Inicializar as variáveis para armazenar as previsões dos modelos
ensemble_preds_best = None
ensemble_preds_last = None
num_models = 0
ser_lab_freq = get_labels_frequency(_csv_path_test, "category", "image_id")
_labels_name = ser_lab_freq.index.values #np.reshape(ser_lab_freq.index.values, (ser_lab_freq.index.values.shape[0], 1))

df_best = pd.DataFrame()
df_last = pd.DataFrame()

# Percorrer as pastas de cada experimento
for experiment_folder in os.listdir(folder_path):

    try:
        if ".csv" in experiment_folder:
            continue

        options = {
            'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average/best'),
            'pred_name_scores': 'predictions.csv',
        }


        experiment_path = os.path.join(folder_path, experiment_folder)

        ensemble_preds_best = None

        experiment_folders = os.listdir(experiment_path)

        # Check if substring "senet154" is present in model_folders
        if any("senet154" in s for s in experiment_folders):
            print("Experimento ", experiment_folder, " não será avaliado")
            continue

        
        print("Experimento: ", experiment_folder)

        
        # Cria path para o experimento atual
        if 'ensemble_average' in experiment_folders:
            experiment_folders.remove('ensemble_average')
        if 'ensemble_3_best_average' in experiment_folders:
            experiment_folders.remove('ensemble_3_best_average')
        else:
            os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average'))
            os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average/best'))

        best_balanced_accuracy = 0
        current_balanced_acuracy = 0
        best_metric_ensemble = None

        # Percorrer as pastas de cada modelo
        for i1 in range(len(experiment_folders)-1):
            for i2 in range(i1+1, len(experiment_folders)-1):
                for i3 in range(i2+1, len(experiment_folders)-1):
                    model_folders = [experiment_folders[i1], experiment_folders[i2], experiment_folders[i3]]

                    metrics_best.options = None

                    for model_folder in model_folders:

                        model_path = os.path.join(experiment_path, model_folder)

                        # Carregar as previsões do modelo atual
                        model_preds_path_best = os.path.join(model_path, "test_pred_best/predictions.csv")

                        model_preds_best = pd.read_csv(model_preds_path_best).sort_values(by=['image'])

                        images_id = model_preds_best[['image']]
                        model_preds_best = model_preds_best[['0','1','2']]

                        # Adicionar as previsões do modelo atual ao ensemble
                        if ensemble_preds_best is None:
                            ensemble_preds_best = model_preds_best
                        else:
                            ensemble_preds_best += model_preds_best

                        num_models += 1

                    # Calcular a média das previsões dos modelos
                    ensemble_preds_best /= num_models
                    
                    # Calcular a balanced accuracy
                    current_balanced_acuracy = cmet.balanced_accuracy(true_labels, ensemble_preds_best.to_numpy())


                    if current_balanced_acuracy > best_balanced_accuracy:
                        best_balanced_accuracy = current_balanced_acuracy

                        best_metric_ensemble = Metrics ("all", _labels_name, options)
                        best_metric_ensemble.update_scores(true_labels.to_numpy(), ensemble_preds_best.to_numpy(), images_id.to_numpy())

                        best_experiment = experiment_folder

                        ensemble_data = {
                            'folder_name': experiment_folder,
                            'ensemble_3_best': model_folders[0].split('fold')[0] + "_" + model_folders[1].split('fold')[0] + "_" + model_folders[2].split('fold')[0]
                        }
                        

                    ensemble_preds_best = None

        best_metric_ensemble.compute_metrics()
        best_metric_ensemble.save_scores()
        
        dict_best = {**ensemble_data, **best_metric_ensemble.metrics_values}

        del dict_best['auc_and_roc_curve']
        del dict_best['precision_recall_report']
        del dict_best['conf_matrix']
        
        df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
    except Exception as e:
        print('Error: ', e)
        continue

df_best.to_csv(os.path.join(folder_path, 'ensemble_3_best_average_best.csv'), index=False)


Experimento  dull_razor_shades_of_gray_cropped_images_folder_2  não será avaliado
Experimento  DA0_noPP  não será avaliado
Experimento:  DA0_PP
Saving the scores in best_results/DA0_PP/ensemble_3_best_average/best
Experimento  dull_razor  não será avaliado
Experimento  DA1_noPP  não será avaliado
Experimento  shades_of_gray_cropped_images_folder  não será avaliado
Experimento  dull_razor_shades_of_gray_cropped_images_folder  não será avaliado
Experimento  DA3_noPP  não será avaliado
Experimento  dull_razor_shades_of_gray  não será avaliado
Experimento:  DA0_noPP_3
Saving the scores in best_results/DA0_noPP_3/ensemble_3_best_average/best
Experimento  dull_razor_2  não será avaliado
Experimento:  DA-2_shades_of_gray_cropped_images_folder_150_epochs
Saving the scores in best_results/DA-2_shades_of_gray_cropped_images_folder_150_epochs/ensemble_3_best_average/best
Experimento:  DA2_cropped_images_folder_150_epochs_lr_init_0001
Saving the scores in best_results/DA2_cropped_images_folder_150

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()   
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)  

# densenet121_fold-1_lrinit-0.0001_batchsize-32_optimizer-Adam_maxepochs-150_DA-2_PPen-None_PPha-None_PPco-shades_of_gray_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_drop-0.0_trainjustclassifier-False_1681078943104982